In [ ]:
googlimport torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR = '/content/drive/My Drive/LAO'

In [ ]:
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

True
cuda


In [ ]:
#Loading MNIST dataset
train_set = torchvision.datasets.MNIST(
    root = BASE_DIR,
    train = True,
    download = False,
    transform = transfroms.Compose([
        transfroms.ToTensor() # Data Normalization
    ])
)
test_set = torchvision.datasets.MNIST(
    root = BASE_DIR,
    train = False,
    download = False,
    transform = transfroms.Compose([
        transfroms.ToTensor() # Data Normalization
    ])
)

In [ ]:
#Hyper Parameters
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 10

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [ ]:
#CNN Model
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()

    self.conv1 = nn.Sequential(
      nn.Conv2d(1, 10, 5, padding = 2),
      nn.BatchNorm2d(10),
      nn.ReLU(inplace=True),
      nn.Conv2d(10, 20, 5),
      nn.BatchNorm2d(20),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(2, 2),
    )

    self.conv2 = nn.Sequential(
      nn.Conv2d(20, 40, 5, padding = 2),
      nn.BatchNorm2d(40),
      nn.ReLU(inplace=True),
      nn.Conv2d(40, 80, 5),
      nn.BatchNorm2d(80),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(2, 2),
    )

    self.fc_net = nn.Sequential(
      nn.Dropout2d(p=0.25, inplace=False),
      nn.Linear(80 * 4 * 4, 100),
      nn.BatchNorm1d(100),
      nn.ReLU(inplace=True),
      nn.Linear(100, num_classes),
    ) # channel * W * H for first linear(FC) parameter

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(-1, 80 * 4 * 4) #2D -> flat
    x = self.fc_net(x)

    return F.log_softmax(x)

model = ConvNet().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Training the CNN Model
for epoch in range(epochs):
    avg_cost = 0

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        hypothesis = model(data)
        cost = criterion(hypothesis, target)
        cost.backward()
        optimizer.step()
        avg_cost += cost / len(train_loader)
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch:    1] cost = 0.127816096
[Epoch:    2] cost = 0.036233101
[Epoch:    3] cost = 0.0260819104
[Epoch:    4] cost = 0.022094816
[Epoch:    5] cost = 0.0189132672
[Epoch:    6] cost = 0.0165449586
[Epoch:    7] cost = 0.0153136784
[Epoch:    8] cost = 0.0136748869
[Epoch:    9] cost = 0.0103081344
[Epoch:   10] cost = 0.00943707395


In [ ]:
#Save model in LOCAL
cwd = os.getcwd()
PATH = os.path.join(cwd,"drive/My Drive/LAO/LAO.pth")
torch.save(model, PATH)

model = torch.load(PATH)
model.eval()

ConvNet(
  (conv1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
    (4): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(20, 40, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(40, 80, kernel_size=(5, 5), stride=(1, 1))
    (4): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_net): Sequential(
    (0): D

In [ ]:
#Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1]
        total += len(target)
        correct += (preds==target).sum().item()
        
    print('Test Accuracy: ', 100.*correct/total, '%')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy:  99.51 %
